In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import os
import shutil
import pickle as pk

from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

from sklearn import metrics

In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/Swarup_Satellite Imaging project/CODES/New Dataset 5000.csv")
data.head(5)

StationId        Datetime  PM2.5  PM10    NO    NO2    NOx    NH3    CO  \
0     TG002  5/14/2018 0:00  35.25  88.5  2.12  33.10  19.30  14.10  0.96   
1     TG002  5/14/2018 1:00  32.00  73.5  2.17  20.30  12.60  13.80  0.17   
2     TG002  5/14/2018 2:00  20.50  58.0  3.00  17.35  11.65  13.48  1.53   
3     TG002  5/14/2018 3:00  21.00  50.0  2.73  18.28  11.93  13.50  2.96   
4     TG002  5/14/2018 4:00  18.50  43.5  2.20  16.38  10.45  15.13  3.49   

    SO2     O3  Benzene  Toluene  Xylene  AQI AQI_Bucket  
0  1.97  67.25     4.98     7.43    2.70  168   Moderate  
1  2.77  80.75     3.40     4.27    1.57  124   Moderate  
2  2.95  77.22     2.02     2.55    1.18  124   Moderate  
3  2.02  36.35     1.42     1.57    0.80  124   Moderate  
4  2.20  70.90     1.20     1.10    1.03  124   Moderate

In [ ]:
df = data.drop(['StationId', 'Datetime', 'AQI_Bucket'], axis=1)
df

PM2.5    PM10     NO     NO2    NOx    NH3    CO   SO2     O3  Benzene  \
0     35.25   88.50   2.12   33.10  19.30  14.10  0.96  1.97  67.25     4.98   
1     32.00   73.50   2.17   20.30  12.60  13.80  0.17  2.77  80.75     3.40   
2     20.50   58.00   3.00   17.35  11.65  13.48  1.53  2.95  77.22     2.02   
3     21.00   50.00   2.73   18.28  11.93  13.50  2.96  2.02  36.35     1.42   
4     18.50   43.50   2.20   16.38  10.45  15.13  3.49  2.20  70.90     1.20   
...     ...     ...    ...     ...    ...    ...   ...   ...    ...      ...   
5086  40.75   93.75   3.55   91.53  51.58  17.98  0.87  3.45  42.12     3.30   
5087  64.25  151.75   4.97  121.20  68.53  21.15  1.21  3.00  13.75     6.22   
5088  82.25  179.50  17.82  142.17  90.10  20.10  1.48  4.05   2.42     8.55   
5089  78.25  165.00  22.60  144.02  94.95  20.75  1.45  3.97   1.20     9.62   
5090  75.50  160.75  12.47  134.22  81.55  21.70  1.29  2.00   4.97     9.65   

      Toluene  Xylene  AQI  
0        7.43    2.70  168  
1        4.27    1.57  124  
2        2.55    1.18  124  
3        1.57    0.80  124  
4        1.10    1.03  124  
...       ...     ...  ...  
5086     7.35    3.00  106  
5087    16.48    8.20  106  
5088    23.40   12.93  106  
5089    24.93   15.65  107  
5090    22.52   15.62  108  

[5091 rows x 13 columns]

In [ ]:
X = df.loc[:, 'NO':'Xylene'].values
y = df['AQI'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf
input_shape=X_train.shape[1]

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_shape=(input_shape,), activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.summary()

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 128)               1408      
                                                                 
 dense_31 (Dense)            (None, 128)               16512     
                                                                 
 dense_32 (Dense)            (None, 1)                 129       
                                                                 
Total params: 18049 (70.50 KB)
Trainable params: 18049 (70.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(X_train, y_train, epochs=1500, validation_data=(X_test, y_test))

Epoch 1/1500
128/128 [==============================] - 1s 5ms/step - loss: 1373.5771 - mae: 25.5402 - val_loss: 507.9975 - val_mae: 17.2905
Epoch 2/1500
128/128 [==============================] - 1s 4ms/step - loss: 488.0058 - mae: 16.5597 - val_loss: 476.6683 - val_mae: 16.5925
Epoch 3/1500
128/128 [==============================] - 0s 4ms/step - loss: 469.5868 - mae: 16.2068 - val_loss: 461.8923 - val_mae: 16.2091
Epoch 4/1500
128/128 [==============================] - 1s 4ms/step - loss: 459.2618 - mae: 16.1249 - val_loss: 455.3707 - val_mae: 16.1811
Epoch 5/1500
128/128 [==============================] - 1s 4ms/step - loss: 454.0602 - mae: 16.0402 - val_loss: 453.3426 - val_mae: 16.1169
Epoch 6/1500
128/128 [==============================] - 1s 5ms/step - loss: 451.8849 - mae: 15.9953 - val_loss: 448.7631 - val_mae: 16.0717
Epoch 7/1500
128/128 [==============================] - 1s 5ms/step - loss: 444.8613 - mae: 15.9036 - val_loss: 447.2415 - val_mae: 16.1550
Epoch 8/1500
128/12

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
pred = model.predict(X_test)

# Plotting the true values vs. predicted values
plt.scatter(y_test, pred, c='blue', label='Actual', alpha=0.5)
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red', label='Perfect Prediction')  # Adding a line for perfect prediction
plt.legend()
plt.show()

32/32 [==============================] - 0s 1ms/step


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, pred)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate R-squared (R2) score
r2 = r2_score(y_test, pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Squared Error (MSE): 456.3023799922803
Mean Absolute Error (MAE): 15.790275834844438
Root Mean Squared Error (RMSE): 21.36123545098177
R-squared (R2) Score: 0.7168613690995216
